<img src="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png" srcset="https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_130 130w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_260 260w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_390 390w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_520 520w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_650 650w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_780 780w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_910 910w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1040 1040w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1170 1170w, https://71022.cdn.cke-cs.com/RructTCFEHceQFc13ldy/images/6dbe93b28dbb43fbc9d50623b68a675a1fedd7608af93b46.png/w_1290 1290w" sizes="100vw" width="1290">

# Advanced Tool Calling with LangChain

This lab demonstrates sophisticated tool calling with LangChain using a room booking system scenario. You'll learn how to implement type-safe tools with proper annotations and compose multiple tools into a cohesive system.

Tasks and details:

* All parts where you should implement something are marked with "..." or a sentence starting with "TASK: "
* You'll implement four tools that work together:
  1. User lookup - Get user details by username
  2. Room listing - Find available rooms by capacity/features
  3. Room booking - Book rooms with conflict checking
  4. Booking history - View user's bookings
* Each tool must be properly type-annotated and include error handling
* The tools will be used by a LangChain agent to handle natural language booking requests

If you need help:
* Use the LangChain chat bot for anything about LangChain: https://chat.langchain.com/
* For general questions you can use ChatGPT or free https://huggingface.co/chat/ with nvidia/Llama-3.1-Nemotron-70B-Instruct-HF LLM - you need a free HuggingFace account to login

### Install required packages

In [ ]:
!pip -q install langchain==0.3.7 langchain-openai==0.2.9 python-dateutil==2.8.2 python-dotenv

### Read environment variables from .env

In [ ]:
%load_ext dotenv
%dotenv

import os

## Tools Implementation

In [ ]:
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Union
from dateutil.parser import parse
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.agents.agent import RunnableMultiActionAgent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Simulated database of users and bookings
USER_DB = {
    "john.doe": {"id": "U123", "name": "John Doe", "department": "Engineering"},
    "jane.smith": {"id": "U456", "name": "Jane Smith", "department": "Marketing"}
}

ROOM_DB = {
    "A101": {"capacity": 4, "features": ["whiteboard", "projector"]},
    "B202": {"capacity": 8, "features": ["whiteboard", "video-conf"]},
    "C303": {"capacity": 12, "features": ["whiteboard", "projector", "video-conf"]}
}

BOOKINGS_DB: List[Dict] = []  # Stores active bookings

# TASK: Implement the get_user_info tool
# This tool should look up a user by username and return their information
@tool
def get_user_info(username: str) -> Union[Dict[str, str], str]:
    """Retrieve user information from the system.

    Args:
        username (str): The username to look up (e.g., 'john.doe')

    Returns:
        Dict[str, str]: User information including ID, name, and department

    Raises:
        ValueError: If the username is not found
    """
    ...

# TASK: Implement the list_available_rooms tool
# This tool should return rooms matching capacity and feature requirements
@tool
def list_available_rooms(capacity: Optional[int] = None, features: Optional[List[str]] = None) -> Union[List[str], str]:
    """List available meeting rooms matching the criteria.

    Args:
        capacity (Optional[int]): Minimum required capacity
        features (Optional[List[str]]): Required features (e.g., ['whiteboard', 'projector'])

    Returns:
        List[str]: List of room IDs matching the criteria
    """
    ...

# TASK: Implement the book_room tool
# This tool should handle room booking with conflict checking
@tool
def book_room(
    user_id: str,
    room_id: str,
    start_time: str,
    duration_minutes: int
) -> Union[Dict[str, Union[str, datetime]], str]:
    """Book a meeting room for a specified time period.

    Args:
        user_id (str): ID of the user making the booking
        room_id (str): ID of the room to book
        start_time (str): Start time in ISO format (YYYY-MM-DD HH:MM)
        duration_minutes (int): Duration of booking in minutes

    Returns:
        Dict[str, Union[str, datetime]]: Booking confirmation with details

    Raises:
        ValueError: If the room is unavailable or inputs are invalid
    """
    ...

# TASK: Implement the get_user_bookings tool
# This tool should return all bookings for a specific user
@tool
def get_user_bookings(user_id: str) -> List[Dict]:
    """Retrieve all bookings for a specific user.

    Args:
        user_id (str): ID of the user to look up bookings for

    Returns:
        List[Dict]: List of booking details for the user
    """
    ...

## Tools "Agent" Setup

In [ ]:
# Use temperature=0 for consistent responses
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# TASK: Provide the tools to the agent
tools = [...]

# TASK: Create a prompt that helps the agent understand its role
prompt = ChatPromptTemplate.from_messages([
    ("system", "..."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])


# TASK: Create the agent and executor
agent = create_openai_tools_agent(...)
agent_executor = AgentExecutor(...)

# We can disbale response streaming like this:
#if isinstance(agent_executor.agent, RunnableMultiActionAgent):
#    agent_executor.agent.stream_runnable = False

## Example Usage

In [ ]:
# Test the booking system with a natural language request
query = "I am jane.smith, and I need to book a room for 6 people with a projector tomorrow at 3pm for 1 hour"

result = agent_executor.invoke({"input": query})
print(result["output"])